# imports

In [ ]:
!pip install torch_geometric
!pip install torch torchvision
!pip install networkx matplotlib
!pip install torch-geometric

In [ ]:
import numpy as np
import networkx as nx

In [ ]:
import torch
import os
import typing
import torch_geometric

import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.datasets as datasets

from torch_geometric.nn import GCNConv, ChebConv
from torch_geometric.nn import global_mean_pool

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx

In [ ]:
import torch
import os
import typing
import torch_geometric

import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.datasets as datasets

from torch_geometric.nn import GCNConv, ChebConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import DataLoader

In [ ]:
if not os.path.exists('dataset/'):
    os.makedirs('dataset/')

# Utils

In [ ]:
def data_load_sep(dataset):
    dataset = dataset.shuffle()

    train_size = int(len(dataset) * 0.8)  # 80% for training
    test_size = len(dataset) - train_size

    train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_features = train_dataset.num_features
    num_classes = train_dataset.num_classes

    return num_features, num_classes,train_loader, test_loader



In [ ]:
def data_stats_node(dataset):
    data = dataset[0]

    # Number of nodes
    num_nodes = data.num_nodes

    # Number of edges
    num_edges = data.num_edges

    # Number of node features
    num_node_features = data.num_features

    # Number of classes
    num_classes = dataset.num_classes

    print("Number of nodes:", num_nodes)
    print("Number of edges:", num_edges)
    print("Number of node features:", num_node_features)
    print("Number of classes:", num_classes)

def data_stats_graph(dataset):
    data = dataset[0]

    num_graphs = len(dataset)

    # Number of node features
    num_node_features = data.num_features

    # Number of classes
    num_classes = dataset.num_classes

    print("Number of graphs:", num_graphs)
    print("Number of node features:", num_node_features)
    print("Number of classes:", num_classes)

In [ ]:
def data_load_sep(dataset):
    dataset = dataset.shuffle()

    train_size = int(len(dataset) * 0.8)  # 80% for training
    test_size = len(dataset) - train_size

    train_dataset, test_dataset = dataset[:train_size], dataset[train_size:]

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    num_features = train_dataset.num_features
    num_classes = train_dataset.num_classes

    return num_features, num_classes,train_loader, test_loader



In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

def plot_graphs_from_dataset(dataset, num_examples=3):
    plt.figure(figsize=(15, 5))

    for i in range(min(num_examples, len(dataset))):
        graph = dataset[i]
        nx_graph = to_networkx(graph, to_undirected=True)

        plt.subplot(1, num_examples, i+1)
        nx.draw(nx_graph, with_labels=True, node_color='skyblue', edge_color='k', node_size=700, font_size=10)
        plt.title(f'Graph {i+1}')

    plt.show()




# Datasets

In [ ]:
from torch_geometric.datasets import MNISTSuperpixels, CitationFull, TUDataset
import numpy as np

In [ ]:
dataset_mnist = MNISTSuperpixels(root='dataset/MNIST/', train=True)


Extracting dataset/MNIST/raw/MNISTSuperpixels.zip
Processing...
Done!


In [ ]:
dataset_MUTAG = TUDataset(root='dataset/MUTAG/', name='MUTAG')
# print("len:", len(dataset_MUTAG))
# print("num_classes:", dataset_MUTAG.num_classes)
# print("num_node_features:", dataset_MUTAG.num_node_features)
data_stats_graph(dataset_MUTAG)

Number of graphs: 188
Number of node features: 7
Number of classes: 2


In [ ]:
dataset_NCI109 = TUDataset(root='dataset/NCI109/', name='NCI109')
# print("len:", len(dataset_NCI109))
# print("num_node_features:", dataset_NCI109.num_node_features)
data_stats_graph(dataset_NCI109)

Number of graphs: 4127
Number of node features: 38
Number of classes: 2


In [ ]:
dataset_NCI1 = TUDataset(root='dataset/NCI1/', name='NCI1')
# print("len:", len(dataset_NCI1))
# print("num_node_features:", dataset_NCI1.num_node_features)
data_stats_graph(dataset_NCI1)

num_classes: 2
Number of graphs: 4110
Number of node features: 37
Number of classes: 2


In [ ]:
from torch_geometric.datasets import TUDataset

dataset_PROTEINS = TUDataset(root='dataset/PROTEINS/', name='PROTEINS')
# print("len:", len(dataset_PROTEINS))
# print("num_classes:", dataset_PROTEINS.num_classes)
# print("num_node_features:", dataset_PROTEINS.num_node_features)
data_stats_graph(dataset_PROTEINS)

Number of graphs: 1113
Number of node features: 3
Number of classes: 2


In [ ]:
dataset_Mutagenicity = TUDataset(root='dataset/Mutagenicity/', name='Mutagenicity')
# print("len:", len(dataset_Mutagenicity))
# print("num_classes:", dataset_Mutagenicity.num_classes)
# print("num_node_features:", dataset_Mutagenicity.num_node_features)
data_stats_graph(dataset_Mutagenicity)

Number of graphs: 4337
Number of node features: 14
Number of classes: 2


# graph classification

## GCN and ChebNet

In [ ]:
#@title Implement GCN

import torch
class GCNModel(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCNModel, self).__init__()
        # BEGIN SOLUTION
        self.layers = nn.ModuleList()
        self.num_features = num_features
        self.num_classes = num_classes

        self.layers.append(GCNConv(num_features, 64))
        self.layers.append(GCNConv(64, 128))
        self.layers.append(GCNConv(128, 128))


          # Output layer
        self.fc = nn.Linear(128, num_classes)

        # END SOLUTION

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # index, weights = get_laplacian(edge_index, normalization = norm)
        # L = to_dense_adj(index, edge_attr=weights, max_num_nodes=graph.num_nodes).squeeze(0)


        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.fc(x)

        # BEGIN SOLUTION

        # END SOLUTION
        return x

In [ ]:
#@title Implement ChebNet

class ChebNetModel(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(ChebNetModel, self).__init__()
        # BEGIN SOLUTION
        self.layers = nn.ModuleList()
        self.num_features = num_features
        self.num_classes = num_classes

        self.layers.append(ChebConv(num_features, 64, K = 3))
        self.layers.append(ChebConv(64, 128, K = 3))
        self.layers.append(ChebConv(128, 128, K = 3))


          # Output layer
        self.fc = nn.Linear(128, num_classes)



        # END SOLUTION

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # BEGIN SOLUTION
        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        # END SOLUTION
        return x

In [ ]:
#@title Training and Testing Loop
def train(model,train_loader, optimizer, epoch=50):
    model.train()
    for epoch in range(epoch):
        losses = 0
        for data in train_loader:
            data = data.to("cuda")
            optimizer.zero_grad()
            out = model(data)
            criterion = torch.nn.CrossEntropyLoss()
            loss = criterion(out, data.y)
            losses += loss.item()
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {losses/len(train_loader)}')

def test(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.to("cuda"))
        pred = out.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

In [ ]:
import torch

def run_experiment(model_class, optimizer_class, num_features, num_classes, train_loader, test_loader, device="cuda", num_runs=5):
    accuracies = []

    for run in range(num_runs):
        # Initialize model
        model = model_class(num_features, num_classes).to(device)

        # Define loss and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optimizer_class(model.parameters(), lr=0.01)

        # Training
        train(model, train_loader, optimizer)

        # Evaluation
        accuracy = test(model, test_loader)
        accuracies.append(accuracy)
        print(f'Run {run + 1}: GCN Accuracy: {accuracy:.3f}')

    # Calculate and print the average accuracy
    avg_accuracy = sum(accuracies) / num_runs
    print(f'Average Accuracy over {num_runs} runs: {avg_accuracy:.3f}')

    return avg_accuracy


In [ ]:
#@title Implement GCN

import torch
class GCNModel(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(GCNModel, self).__init__()
        # BEGIN SOLUTION
        self.layers = nn.ModuleList()
        self.num_features = num_features
        self.num_classes = num_classes

        self.layers.append(GCNConv(num_features, 64))
        self.layers.append(GCNConv(64, 128))
        self.layers.append(GCNConv(128, 128))


          # Output layer
        self.fc = nn.Linear(128, num_classes)

        # END SOLUTION

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # index, weights = get_laplacian(edge_index, normalization = norm)
        # L = to_dense_adj(index, edge_attr=weights, max_num_nodes=graph.num_nodes).squeeze(0)


        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.fc(x)

        # BEGIN SOLUTION

        # END SOLUTION
        return x

In [ ]:
#@title Implement ChebNet

class ChebNetModel(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(ChebNetModel, self).__init__()
        # BEGIN SOLUTION
        self.layers = nn.ModuleList()
        self.num_features = num_features
        self.num_classes = num_classes

        self.layers.append(ChebConv(num_features, 64, K = 3))
        self.layers.append(ChebConv(64, 128, K = 3))
        self.layers.append(ChebConv(128, 128, K = 3))


          # Output layer
        self.fc = nn.Linear(128, num_classes)



        # END SOLUTION

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # BEGIN SOLUTION
        for layer in self.layers:
            x = layer(x, edge_index)
            x = F.relu(x)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        # END SOLUTION
        return x

In [ ]:
#@title Training and Testing Loop
def train(model,train_loader, optimizer, epoch=50):
    model.train()
    for epoch in range(epoch):
        losses = 0
        for data in train_loader:
            data = data.to("cuda")
            optimizer.zero_grad()
            out = model(data)
            criterion = torch.nn.CrossEntropyLoss()
            loss = criterion(out, data.y)
            losses += loss.item()
            loss.backward()
            optimizer.step()
        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {losses/len(train_loader)}')

def test(model, loader):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.to("cuda"))
        pred = out.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
    return correct / len(loader.dataset)

In [ ]:
import torch

# Assuming the GCNModel and ChebNetModel classes, train, and test functions are defined above
# Also assuming train_loader and test_loader are defined

def run_experiment(model_class, optimizer_class, num_features, num_classes, train_loader, test_loader, device="cuda", num_runs=5):
    accuracies = []

    for run in range(num_runs):
        # Initialize model
        model = model_class(num_features, num_classes).to(device)

        # Define loss and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = optimizer_class(model.parameters(), lr=0.01)

        # Training
        train(model, train_loader, optimizer)

        # Evaluation
        accuracy = test(model, test_loader)
        accuracies.append(accuracy)
        print(f'Run {run + 1}: GCN Accuracy: {accuracy:.3f}')

    # Calculate and print the average accuracy
    avg_accuracy = sum(accuracies) / num_runs
    print(f'Average Accuracy over {num_runs} runs: {avg_accuracy:.3f}')

    return avg_accuracy


# Experiments

## MUTAG

In [ ]:
n_features_mutag, n_class_mutag, train_mutag, test_mutag = data_load_sep(dataset_MUTAG)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
avg_gcn_mutag = run_experiment(GCNModel, torch.optim.Adam, n_features_mutag, n_class_mutag, train_mutag, test_mutag, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6861956119537354
Epoch 10, Loss: 0.560483455657959
Epoch 20, Loss: 0.5307190954685211
Epoch 30, Loss: 0.5079348862171174
Epoch 40, Loss: 0.5218905210494995
Run 1: GCN Accuracy: 0.84
Epoch 0, Loss: 0.6652185320854187
Epoch 10, Loss: 0.5361934423446655
Epoch 20, Loss: 0.531353098154068
Epoch 30, Loss: 0.5276401698589325
Epoch 40, Loss: 0.5164557814598083
Run 2: GCN Accuracy: 0.82
Epoch 0, Loss: 0.7081715345382691
Epoch 10, Loss: 0.5577500998973847
Epoch 20, Loss: 0.5205749869346619
Epoch 30, Loss: 0.5039950489997864
Epoch 40, Loss: 0.4872865557670593
Run 3: GCN Accuracy: 0.84
Epoch 0, Loss: 0.6815659880638123
Epoch 10, Loss: 0.5447292447090148
Epoch 20, Loss: 0.5321837604045868
Epoch 30, Loss: 0.5021425485610962
Epoch 40, Loss: 0.5270025908946991
Run 4: GCN Accuracy: 0.84
Epoch 0, Loss: 0.6719547510147095
Epoch 10, Loss: 0.5531218647956848
Epoch 20, Loss: 0.5167851209640503
Epoch 30, Loss: 0.5135402500629425
Epoch 40, Loss: 0.5159900605678558
Run 5: GCN Accuracy: 0.82
Av

In [ ]:
avg_cheb_mutag = run_experiment(ChebNetModel, torch.optim.Adam, n_features_mutag, n_class_mutag, train_mutag, test_mutag, device="cuda", num_runs=5)

Epoch 0, Loss: 0.747893238067627
Epoch 10, Loss: 0.5424605906009674
Epoch 20, Loss: 0.4802164971828461
Epoch 30, Loss: 0.3948553383350372
Epoch 40, Loss: 0.36535066962242124
Run 1: GCN Accuracy: 0.89
Epoch 0, Loss: 0.7632253646850586
Epoch 10, Loss: 0.523601233959198
Epoch 20, Loss: 0.45214914679527285
Epoch 30, Loss: 0.4127261698246002
Epoch 40, Loss: 0.3367288649082184
Run 2: GCN Accuracy: 0.87
Epoch 0, Loss: 0.8166373014450073
Epoch 10, Loss: 0.5091387033462524
Epoch 20, Loss: 0.4493148148059845
Epoch 30, Loss: 0.36430134773254397
Epoch 40, Loss: 0.34980141520500185
Run 3: GCN Accuracy: 0.89
Epoch 0, Loss: 0.7903151273727417
Epoch 10, Loss: 0.5362716972827911
Epoch 20, Loss: 0.4607440888881683
Epoch 30, Loss: 0.3573861360549927
Epoch 40, Loss: 0.34974956810474395
Run 4: GCN Accuracy: 0.95
Epoch 0, Loss: 0.7846099853515625
Epoch 10, Loss: 0.507878977060318
Epoch 20, Loss: 0.4310144305229187
Epoch 30, Loss: 0.34325504302978516
Epoch 40, Loss: 0.4512578994035721
Run 5: GCN Accuracy: 0.

## NCI109

In [ ]:
n_features_n109, n_class_n109, train_n109, test_n109 = data_load_sep(dataset_NCI109)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
avg_gcn_n109 = run_experiment(GCNModel, torch.optim.Adam,n_features_n109, n_class_n109, train_n109, test_n109, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6889100023187124
Epoch 10, Loss: 0.6240652275200074
Epoch 20, Loss: 0.6043148470612673
Epoch 30, Loss: 0.5978447823570325
Epoch 40, Loss: 0.5869097583569013
Run 1: GCN Accuracy: 0.69
Epoch 0, Loss: 0.6906875056716112
Epoch 10, Loss: 0.6328973658382893
Epoch 20, Loss: 0.6028035002259108
Epoch 30, Loss: 0.596096792186682
Epoch 40, Loss: 0.5839785781617348
Run 2: GCN Accuracy: 0.69
Epoch 0, Loss: 0.6893774379904454
Epoch 10, Loss: 0.6218718863450564
Epoch 20, Loss: 0.6114021809055254
Epoch 30, Loss: 0.5869417789463813
Epoch 40, Loss: 0.5787126172620517
Run 3: GCN Accuracy: 0.69
Epoch 0, Loss: 0.6901538245953046
Epoch 10, Loss: 0.666202707359424
Epoch 20, Loss: 0.6163995853410318
Epoch 30, Loss: 0.6079001567111566
Epoch 40, Loss: 0.5967410785647539
Run 4: GCN Accuracy: 0.70
Epoch 0, Loss: 0.6928844514947671
Epoch 10, Loss: 0.6148749675888282
Epoch 20, Loss: 0.6010967808274122
Epoch 30, Loss: 0.591618648228737
Epoch 40, Loss: 0.5900312579022005
Run 5: GCN Accuracy: 0.69
Ave

In [ ]:
avg_cheb_n109 = run_experiment(ChebNetModel, torch.optim.Adam, n_features_n109, n_class_n109, train_n109, test_n109, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6856988977927428
Epoch 10, Loss: 0.5312940971209452
Epoch 20, Loss: 0.4407740958894675
Epoch 30, Loss: 0.32005633120066845
Epoch 40, Loss: 0.26823130070876616
Run 1: GCN Accuracy: 0.74
Epoch 0, Loss: 0.6882074223114893
Epoch 10, Loss: 0.5133674179132168
Epoch 20, Loss: 0.39222512236581397
Epoch 30, Loss: 0.2839719755575061
Epoch 40, Loss: 0.18741728000056285
Run 2: GCN Accuracy: 0.77
Epoch 0, Loss: 0.6965958693852792
Epoch 10, Loss: 0.5291001805319235
Epoch 20, Loss: 0.3947339573731789
Epoch 30, Loss: 0.3013307796791196
Epoch 40, Loss: 0.22553970442654994
Run 3: GCN Accuracy: 0.74
Epoch 0, Loss: 0.6951030257802743
Epoch 10, Loss: 0.5258718654513359
Epoch 20, Loss: 0.4170176772257456
Epoch 30, Loss: 0.3214774549437257
Epoch 40, Loss: 0.25881220020640355
Run 4: GCN Accuracy: 0.77
Epoch 0, Loss: 0.6967258831629386
Epoch 10, Loss: 0.5259193276556638
Epoch 20, Loss: 0.4158511573018936
Epoch 30, Loss: 0.3057875318381076
Epoch 40, Loss: 0.24308490094084007
Run 5: GCN Accuracy

## NCI1

In [ ]:
n_features_n1, n_class_n1, train_n1, test_n1 = data_load_sep(dataset_NCI1)

In [ ]:
avg_gcn_n1 = run_experiment(GCNModel, torch.optim.Adam,n_features_n1, n_class_n1, train_n1, test_n1, device="cuda", num_runs=5)

Epoch 0, Loss: 0.688850601899971
Epoch 10, Loss: 0.6188836277109905
Epoch 20, Loss: 0.5983244082302723
Epoch 30, Loss: 0.5925686240774913
Epoch 40, Loss: 0.5822236572654502
Run 1: GCN Accuracy: 0.68
Epoch 0, Loss: 0.6886810938131462
Epoch 10, Loss: 0.6085108110626924
Epoch 20, Loss: 0.5967187424307888
Epoch 30, Loss: 0.5884608371743878
Epoch 40, Loss: 0.5777030708720383
Run 2: GCN Accuracy: 0.68
Epoch 0, Loss: 0.6816946511129731
Epoch 10, Loss: 0.6203866884546373
Epoch 20, Loss: 0.5999459713986777
Epoch 30, Loss: 0.5769898816219812
Epoch 40, Loss: 0.5750586679838237
Run 3: GCN Accuracy: 0.70
Epoch 0, Loss: 0.6906032012504282
Epoch 10, Loss: 0.6401298676879661
Epoch 20, Loss: 0.6024148472883168
Epoch 30, Loss: 0.5878982344298687
Epoch 40, Loss: 0.5811679834879718
Run 4: GCN Accuracy: 0.67
Epoch 0, Loss: 0.690982465605134
Epoch 10, Loss: 0.6493744983256442
Epoch 20, Loss: 0.6044497116676812
Epoch 30, Loss: 0.5899141850402054
Epoch 40, Loss: 0.5827876066119926
Run 5: GCN Accuracy: 0.69
Av

In [ ]:
avg_cheb_n1 = run_experiment(ChebNetModel, torch.optim.Adam, n_features_n1, n_class_n1, train_n1, test_n1, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6859373913227933
Epoch 10, Loss: 0.4852144959771517
Epoch 20, Loss: 0.3614804134930222
Epoch 30, Loss: 0.2469374016650672
Epoch 40, Loss: 0.18037918260664615
Run 1: GCN Accuracy: 0.77
Epoch 0, Loss: 0.6783451125459764
Epoch 10, Loss: 0.4743115563415787
Epoch 20, Loss: 0.3540169654540645
Epoch 30, Loss: 0.2748041959497535
Epoch 40, Loss: 0.20015541257933506
Run 2: GCN Accuracy: 0.80
Epoch 0, Loss: 0.6883385754326015
Epoch 10, Loss: 0.49134843181637883
Epoch 20, Loss: 0.3935285246777303
Epoch 30, Loss: 0.3008260920788478
Epoch 40, Loss: 0.21312369373672216
Run 3: GCN Accuracy: 0.79
Epoch 0, Loss: 0.6711140546983886
Epoch 10, Loss: 0.46553213648425723
Epoch 20, Loss: 0.37451668503215013
Epoch 30, Loss: 0.2712883118142202
Epoch 40, Loss: 0.21530513649860633
Run 4: GCN Accuracy: 0.78
Epoch 0, Loss: 0.6868152444802441
Epoch 10, Loss: 0.4940433264936058
Epoch 20, Loss: 0.3816596959690446
Epoch 30, Loss: 0.30508693662083264
Epoch 40, Loss: 0.21222038114967856
Run 5: GCN Accura

## Protein

In [ ]:
n_features_pro, n_class_pro, train_pro, test_pro = data_load_sep(dataset_PROTEINS)

In [ ]:
avg_gcn_pro = run_experiment(GCNModel, torch.optim.Adam,n_features_pro, n_class_pro, train_pro, test_pro, device="cuda", num_runs=5)

Epoch 0, Loss: 0.670258526291166
Epoch 10, Loss: 0.6069668424980981
Epoch 20, Loss: 0.6013015263846943
Epoch 30, Loss: 0.5949149940695081
Epoch 40, Loss: 0.5978584566286632
Run 1: GCN Accuracy: 0.70
Epoch 0, Loss: 0.6722206345626286
Epoch 10, Loss: 0.6176268884113857
Epoch 20, Loss: 0.6075559939656939
Epoch 30, Loss: 0.6110151048217501
Epoch 40, Loss: 0.5973752353872571
Run 2: GCN Accuracy: 0.70
Epoch 0, Loss: 0.6706650406122208
Epoch 10, Loss: 0.6068844071456364
Epoch 20, Loss: 0.5994606550250735
Epoch 30, Loss: 0.5894094418202128
Epoch 40, Loss: 0.5865684204867908
Run 3: GCN Accuracy: 0.70
Epoch 0, Loss: 0.6670129575899669
Epoch 10, Loss: 0.6193076893687248
Epoch 20, Loss: 0.5959215472851481
Epoch 30, Loss: 0.5970521079642432
Epoch 40, Loss: 0.5918915026954242
Run 4: GCN Accuracy: 0.69
Epoch 0, Loss: 0.679140003664153
Epoch 10, Loss: 0.6106728113123349
Epoch 20, Loss: 0.5928194522857666
Epoch 30, Loss: 0.6006427205034665
Epoch 40, Loss: 0.596614135163171
Run 5: GCN Accuracy: 0.71
Ave

In [ ]:
avg_cheb_pro = run_experiment(ChebNetModel, torch.optim.Adam, n_features_pro, n_class_pro, train_pro, test_pro, device="cuda", num_runs=5)

Epoch 0, Loss: 0.7078274616173336
Epoch 10, Loss: 0.5885907613805362
Epoch 20, Loss: 0.5621591455170086
Epoch 30, Loss: 0.5539234578609467
Epoch 40, Loss: 0.5241678176181657
Run 1: GCN Accuracy: 0.67
Epoch 0, Loss: 0.6907059933458056
Epoch 10, Loss: 0.5884103338633265
Epoch 20, Loss: 0.5652870535850525
Epoch 30, Loss: 0.5295917551432338
Epoch 40, Loss: 0.4884109582219805
Run 2: GCN Accuracy: 0.70
Epoch 0, Loss: 0.680924973317555
Epoch 10, Loss: 0.6064405164548329
Epoch 20, Loss: 0.5499961088810649
Epoch 30, Loss: 0.5532949577484813
Epoch 40, Loss: 0.5292375705071858
Run 3: GCN Accuracy: 0.74
Epoch 0, Loss: 0.6970781832933426
Epoch 10, Loss: 0.607182397374085
Epoch 20, Loss: 0.5697032149348941
Epoch 30, Loss: 0.547849283686706
Epoch 40, Loss: 0.5147132671305111
Run 4: GCN Accuracy: 0.75
Epoch 0, Loss: 0.7219081521034241
Epoch 10, Loss: 0.5947386739509446
Epoch 20, Loss: 0.5544306284614972
Epoch 30, Loss: 0.5293855017849377
Epoch 40, Loss: 0.4950957936900003
Run 5: GCN Accuracy: 0.71
Ave

## Mutagenicity

In [ ]:
n_features_mcity, n_class_mcity, train_mcity, test_mcity = data_load_sep(dataset_Mutagenicity)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
av_gcn_mcity = run_experiment(GCNModel, torch.optim.Adam,n_features_mcity, n_class_mcity, train_mcity, test_mcity, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6209412241747619
Epoch 10, Loss: 0.5128102414651748
Epoch 20, Loss: 0.48347938115443656
Epoch 30, Loss: 0.46878937187544795
Epoch 40, Loss: 0.45869048117497646
Run 1: GCN Accuracy: 0.78
Epoch 0, Loss: 0.6340817470069325
Epoch 10, Loss: 0.5108232186474931
Epoch 20, Loss: 0.4863740875633485
Epoch 30, Loss: 0.47517187825036705
Epoch 40, Loss: 0.45952998006015744
Run 2: GCN Accuracy: 0.77
Epoch 0, Loss: 0.6288673076607766
Epoch 10, Loss: 0.5258376472586885
Epoch 20, Loss: 0.4870926018701781
Epoch 30, Loss: 0.460318576852116
Epoch 40, Loss: 0.44859400938410277
Run 3: GCN Accuracy: 0.78
Epoch 0, Loss: 0.6181474251484652
Epoch 10, Loss: 0.5102562827801486
Epoch 20, Loss: 0.4833223036396394
Epoch 30, Loss: 0.47012958791824655
Epoch 40, Loss: 0.4591825296846005
Run 4: GCN Accuracy: 0.79
Epoch 0, Loss: 0.6155925305611497
Epoch 10, Loss: 0.5253445897080483
Epoch 20, Loss: 0.48969213268078793
Epoch 30, Loss: 0.4691074604014738
Epoch 40, Loss: 0.4536456935996309
Run 5: GCN Accuracy

In [ ]:
avg_cheb_mcity = run_experiment(ChebNetModel, torch.optim.Adam, n_features_mcity, n_class_mcity, train_mcity, test_mcity, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6404408022351221
Epoch 10, Loss: 0.425510907938721
Epoch 20, Loss: 0.334379019950508
Epoch 30, Loss: 0.2721099368202577
Epoch 40, Loss: 0.2364498694692183
Run 1: GCN Accuracy: 0.82
Epoch 0, Loss: 0.6140541335858336
Epoch 10, Loss: 0.41223735464822264
Epoch 20, Loss: 0.3483977350619955
Epoch 30, Loss: 0.28306814693256255
Epoch 40, Loss: 0.23413253807966863
Run 2: GCN Accuracy: 0.80
Epoch 0, Loss: 0.6207321267062371
Epoch 10, Loss: 0.42272596252620764
Epoch 20, Loss: 0.34841401710969594
Epoch 30, Loss: 0.2958666491194056
Epoch 40, Loss: 0.24518475291925831
Run 3: GCN Accuracy: 0.82
Epoch 0, Loss: 0.6322274101436685
Epoch 10, Loss: 0.41829968551430136
Epoch 20, Loss: 0.3575109198279337
Epoch 30, Loss: 0.3032654496507907
Epoch 40, Loss: 0.2600628776014398
Run 4: GCN Accuracy: 0.83
Epoch 0, Loss: 0.6277538417676173
Epoch 10, Loss: 0.4258471866813275
Epoch 20, Loss: 0.32799501026715705
Epoch 30, Loss: 0.2622071011760913
Epoch 40, Loss: 0.20202690280905558
Run 5: GCN Accuracy

## MNIST

In [ ]:
n_features_mnist, n_class_mnist, train_mnist, test_mnist = data_load_sep(dataset_mnist)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
av_gcn_mnist = run_experiment(GCNModel, torch.optim.Adam,n_features_mnist, n_class_mnist, train_mnist, test_mnist, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6209412241747619
Epoch 10, Loss: 0.5128102414651748
Epoch 20, Loss: 0.48347938115443656
Epoch 30, Loss: 0.46878937187544795
Epoch 40, Loss: 0.45869048117497646
Run 1: GCN Accuracy: 0.78
Epoch 0, Loss: 0.6340817470069325
Epoch 10, Loss: 0.5108232186474931
Epoch 20, Loss: 0.4863740875633485
Epoch 30, Loss: 0.47517187825036705
Epoch 40, Loss: 0.45952998006015744
Run 2: GCN Accuracy: 0.77
Epoch 0, Loss: 0.6288673076607766
Epoch 10, Loss: 0.5258376472586885
Epoch 20, Loss: 0.4870926018701781
Epoch 30, Loss: 0.460318576852116
Epoch 40, Loss: 0.44859400938410277
Run 3: GCN Accuracy: 0.78
Epoch 0, Loss: 0.6181474251484652
Epoch 10, Loss: 0.5102562827801486
Epoch 20, Loss: 0.4833223036396394
Epoch 30, Loss: 0.47012958791824655
Epoch 40, Loss: 0.4591825296846005
Run 4: GCN Accuracy: 0.79
Epoch 0, Loss: 0.6155925305611497
Epoch 10, Loss: 0.5253445897080483
Epoch 20, Loss: 0.48969213268078793
Epoch 30, Loss: 0.4691074604014738
Epoch 40, Loss: 0.4536456935996309
Run 5: GCN Accuracy

In [ ]:
avg_cheb_mnist = run_experiment(ChebNetModel, torch.optim.Adam, n_features_mnist, n_class_mnist, train_mnist, test_mnist, device="cuda", num_runs=5)

Epoch 0, Loss: 0.6404408022351221
Epoch 10, Loss: 0.425510907938721
Epoch 20, Loss: 0.334379019950508
Epoch 30, Loss: 0.2721099368202577
Epoch 40, Loss: 0.2364498694692183
Run 1: GCN Accuracy: 0.82
Epoch 0, Loss: 0.6140541335858336
Epoch 10, Loss: 0.41223735464822264
Epoch 20, Loss: 0.3483977350619955
Epoch 30, Loss: 0.28306814693256255
Epoch 40, Loss: 0.23413253807966863
Run 2: GCN Accuracy: 0.80
Epoch 0, Loss: 0.6207321267062371
Epoch 10, Loss: 0.42272596252620764
Epoch 20, Loss: 0.34841401710969594
Epoch 30, Loss: 0.2958666491194056
Epoch 40, Loss: 0.24518475291925831
Run 3: GCN Accuracy: 0.82
Epoch 0, Loss: 0.6322274101436685
Epoch 10, Loss: 0.41829968551430136
Epoch 20, Loss: 0.3575109198279337
Epoch 30, Loss: 0.3032654496507907
Epoch 40, Loss: 0.2600628776014398
Run 4: GCN Accuracy: 0.83
Epoch 0, Loss: 0.6277538417676173
Epoch 10, Loss: 0.4258471866813275
Epoch 20, Loss: 0.32799501026715705
Epoch 30, Loss: 0.2622071011760913
Epoch 40, Loss: 0.20202690280905558
Run 5: GCN Accuracy